First import all the necessary files

In [1]:
import numpy as np 
from scipy.io import loadmat
import sklearn
import tensorflow as tf
import matplotlib.pyplot as plt
from string import ascii_lowercase
from random import randint
import sys,os

tf.logging.set_verbosity(tf.logging.INFO)
os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

DATASET = "small" # or "large"


/home/frunkad/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mat_file = loadmat("datasets/notMNIST_small.mat")
L = len(mat_file['labels'])

mat_file['images'] = np.array([mat_file['images'][:,:,i] for i in range(L)])
# vf = np.vectorize(lambda x: ascii_lowercase[int(x)])
vf = np.vectorize(lambda x: int(x))
mat_file['labels'] = vf(mat_file['labels'])

seed = randint(1,1000)
np.random.seed(seed)
np.random.shuffle(mat_file['images'])
np.random.seed(seed)
np.random.shuffle(mat_file['labels'])

test_len = L//4
train_len = L - test_len
trainX = mat_file['images'][test_len:]
trainy = mat_file['labels'][test_len:]
testX = mat_file['images'][:test_len]
testy = mat_file['labels'][:test_len]

In [3]:
print("Training Images: {}\nTesting Images: {}".format(train_len,test_len))
trainX /= 255
testX /= 255


Training Images: 14043
Testing Images: 4681


In [4]:

def tostr(s):
    if callable(s):
        return s.__name__
    else:
        return str(s)

def create_networks():
    global trainX,testX,testy,trainy
#     trainX = np.reshape(trainX,(train_len,28*28))
#     testX = np.reshape(testX,(test_len,28*28))
    trainy = tf.keras.utils.to_categorical(trainy,10)
    testy = tf.keras.utils.to_categorical(testy,10)


    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu, input_shape=(784,)))
    model.add(tf.keras.layers.Dropout(0.65))
    model.add(tf.keras.layers.Dense(10,activation=tf.nn.softmax))
    # 
    # We will now compile and print out a summary of our model
    model.compile(loss='categorical_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])
    return model

def try_model(
                epochs = 5,
                save = False,
                batch_size = None,
                model = create_networks()
            ):
    try_model.count += 1


    
    model.fit(trainX,trainy,epochs=epochs,verbose=1,batch_size=batch_size)
    lossed,accuracy = model.evaluate(testX,testy,verbose=1)
    print("====================================\nModel ",try_model.count," : ")
    # print("\tActivator 1: {}\n\tActivator 2: {}\n\tLoss: {}\n\tOptimizer: {}\n\tEpochs: {}".format(tostr(activator1),tostr(activator2),loss,tostr(optimizer),epochs))
    model.summary()
    print(bcolors.OKGREEN+"\tAccuracy: {}".format(accuracy)+bcolors.ENDC)
    print(bcolors.WARNING+"\tLoss: {}".format(lossed)+bcolors.ENDC)
    if save:
        tf.keras.models.save_model(model,"model-"+DATASET+"-"+time.ctime()+".hdf5")

try_model.count = 0

In [5]:
# try_model()

In [6]:
trainX = np.expand_dims(trainX,-1)
testX = np.expand_dims(trainX,-1)


trainy = tf.keras.utils.to_categorical(trainy,10)
testy = tf.keras.utils.to_categorical(testy,10)

In [7]:
img_input = tf.keras.layers.Input(shape=(28, 28, 1))

# First convolution extracts 16 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = tf.keras.layers.Conv2D(16, 3, activation='relu')(img_input)
x = tf.keras.layers.MaxPooling2D(2)(x)

# Second convolution extracts 32 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = tf.keras.layers.Conv2D(32, 3, activation='relu')(x)
x = tf.keras.layers.MaxPooling2D(2)(x)

# Third convolution extracts 64 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = tf.keras.layers.Conv2D(64, 3, activation='relu')(x)
x = tf.keras.layers.MaxPooling2D(2)(x)

In [11]:
x = tf.keras.layers.Flatten()(x)

# Create a fully connected layer with ReLU activation and 512 hidden units
x = tf.keras.layers.Dense(512, activation='relu')(x)

# Create output layer with a single node and sigmoid activation
output = tf.keras.layers.Dense(10, activation='sigmoid')(x)

# Create model:
# input = input feature map
# output = input feature map + stacked convolution/maxpooling layers + fully 
# connected layer + sigmoid output layer
model = tf.keras.models.Model(img_input, output)

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(lr=0.001),
              metrics=['accuracy'])
model.summary()
try_model(model=model)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 1, 1, 64)          0         
__________

ValueError: Error when checking target: expected dense_10 to have 2 dimensions, but got array with shape (14043, 10, 10)